In [ ]:
import threading

condition = threading.Condition()
item_available = False

def consumer():
    with condition:
        if not item_available:
            print("Consumer waiting for item")
            condition.wait()
        print("Consumer consumed the item")

def producer():
    global item_available
    with condition:
        print("Producer produced an item")
        item_available = True
        condition.notify()

def run():
    producer_thread = threading.Thread(target=producer)
    consumer_thread = threading.Thread(target=consumer)

    consumer_thread.start()
    producer_thread.start()

    producer_thread.join()
    consumer_thread.join()

run()

#### Explanation

1. **Initial Check**: When the consumer() function is called, it first acquires the condition lock (because of the with condition: statement).

2. **Waiting for the Condition**:
    - The `if not item_available`: check is used to determine if the item is available for consumption.
    - If `item_available` is `False` (meaning the item is not available), the consumer calls condition.wait(). This does two things:
      - Releases the lock temporarily. This allows other threads (like the producer) to acquire the lock and change the state (i.e., produce an item).
      - Blocks (waits) until `condition.notify()` or `condition.notify_all()` is called by another thread (the producer in this case).
  
3. **Resuming After Notification**:
   - Once the producer calls `condition.notify()`, the consumer is awakened (unblocked) and re-acquires the lock.
   - After re-acquiring the lock, it proceeds to the next line and completes its execution.